In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2DTranspose

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
# from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import tensorflow as tf

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [ ]:

# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(F2,kernel_size=(f,f),strides=(1,1),padding='same',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3,(1,1),strides=(1,1),padding='valid',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=-1)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [ ]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters =F1, kernel_size =(1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters =F2, kernel_size =(f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters =F3, kernel_size =(1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters =F3, kernel_size =(1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut]) 
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [ ]:

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 1)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 1)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
   
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (240, 320, 3))
print(model.output_shape)

(None, 30, 40, 2048)


In [ ]:
# loading resnet50 model using tensorflow
model2=tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(240,320,3), pooling=None, 
)

In [ ]:
for i in range(1,len(model.layers)):
#     print(i, model.layers[i].output_shape, model2.layers[i].output_shape)
#     print(model.layers[i],model2.layers[i])
    try:
        weights = model2.layers[i].get_weights()
        model.layers[i].set_weights(weights)
        print(i)
    except Exception as e:
        print("pass",i)
        pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
pass 45
pass 46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
pass 87
pass 88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
pass 149
pass 150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174


In [ ]:
class LSU(tf.keras.layers.Layer):
    def __init__(self, kernal, unit = 32):
        super(LSU, self).__init__()
        self.unit = unit
        self.kernal = kernal
    
    def build(self, kernal):
        self.kernal_d1 = self.add_weight(shape=(self.kernal[-1], self.unit), dtype = tf.float32, name = 'alpha')
        self.kernal_s1 = self.add_weight(shape=(self.kernal[-1], self.unit), dtype = tf.float32, name = 'gamma')
    
    def call(self, input_tensor):
        layer = (input_tensor[0] * self.kernal_d1) + (input_tensor[1] * self.kernal_s1)
        return layer
      
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'unit': self.unit,
            'kernal': self.kernal})
        #     'units': self.units,
        #     'd_model': self.d_model,
        #     'num_heads': self.num_heads,
        #     'dropout': self.dropout,
        # })
        return config

In [ ]:
x1_depth=tf.keras.layers.Conv2DTranspose(512,(2,2),name='conv1_d',activation='relu', strides = 2)(model.output)

x1_semantic=tf.keras.layers.Conv2DTranspose(512,(2,2),name='conv1_s',activation='relu', strides = 2)(model.output)

x2_depth=tf.keras.layers.Conv2DTranspose(128,(2,2), strides = 2, name='conv2_d',activation='relu')(x1_semantic)

x2_semantic=tf.keras.layers.Conv2DTranspose(128,(2,2), strides = 2, name='conv2_s',activation='relu')(x1_semantic)

layer_1 = LSU(kernal = (1,1), unit = 512)
lsu_1 = layer_1([x1_depth, x1_semantic])
lsu_1 = tf.keras.layers.Conv2DTranspose(x2_depth.shape[-1], (2,2), strides = 2, name = 'lsu_1_t', activation = 'relu')(lsu_1)

x3_depth_input = tf.keras.layers.Add(name = 'x3_d_input')([x2_depth, lsu_1])
x3_semantic_input = tf.keras.layers.Add(name = 'x3_s_input')([x2_semantic, lsu_1])

x3_depth=tf.keras.layers.Conv2DTranspose(64,(2,2),strides = 2,name='conv3_d',activation='relu')(x3_depth_input)

x3_semantic=tf.keras.layers.Conv2DTranspose(64,(2,2),strides = 2,name='conv3_s',activation='relu')(x3_semantic_input)

layer_2 = LSU(kernal = (1,1), unit = 128)
lsu_2 = layer_2([x2_depth, x2_semantic])
lsu_2 = tf.keras.layers.Conv2DTranspose(x3_depth.shape[-1], (2,2), strides = 2, name = 'lsu_2_t', activation = 'relu')(lsu_2)

x4_depth_input = tf.keras.layers.Add(name = 'x4_d_input')([x3_depth, lsu_2])
x4_semantic_input = tf.keras.layers.Add(name = 'x4_s_input')([x3_semantic, lsu_2])

x4_depth = tf.keras.layers.Conv2DTranspose(1,(2,2),strides = 2, name="conv4_d",activation='relu')(x4_depth_input)

x4_semantic = tf.keras.layers.Conv2DTranspose(3,(2,2),strides = 2, name="conv4_s",activation='relu')(x4_semantic_input)



'''
  Apply semi dense up skip connection between f_in, f1, f2, f3 layer
  f_out=h(f_in)+sumation_of_(h(f(i))) where i belong to (1,n)
'''
#add_shape  of f4 output
x4_semantic_shape_x_y = (240,320)
x4_semantic_filters=x4_semantic.shape[-1]

# BASE INPUT LAYER UPSAMPLING
bilinear_upsampling_in = tf.keras.layers.experimental.preprocessing.Resizing(x4_semantic_shape_x_y[0], x4_semantic_shape_x_y[1], interpolation='bilinear')(model.output)
x_semantic_out_in=tf.keras.layers.Conv2D(x4_semantic_filters,(1,1),name='resize_filter_conv_in_s',activation='relu')(bilinear_upsampling_in)

# FIRST LAYER BILINEAR UPsAMPLING
bilinear_upsampling_1 = tf.keras.layers.experimental.preprocessing.Resizing(x4_semantic_shape_x_y[0], x4_semantic_shape_x_y[1], interpolation='bilinear')(x1_semantic)
x_semantic_out1 = tf.keras.layers.Conv2D(x4_semantic_filters,(1,1),name='resize_filter_conv1_s',activation='relu')(bilinear_upsampling_1)

#SECOND LAYER BILINEAR UPSAMPLING
bilinear_upsampling_2 = tf.keras.layers.experimental.preprocessing.Resizing(x4_semantic_shape_x_y[0], x4_semantic_shape_x_y[1], interpolation='bilinear')(x2_semantic)
x_semantic_out2 = tf.keras.layers.Conv2D(x4_semantic_filters,(1,1),name='resize_filter_conv2_s',activation='relu')(bilinear_upsampling_2)

# 3rd LAYER UPSAMPLING
bilinear_upsampling_3 = tf.keras.layers.experimental.preprocessing.Resizing(x4_semantic_shape_x_y[0], x4_semantic_shape_x_y[1], interpolation='bilinear')(x3_semantic)
x_semantic_out3 = tf.keras.layers.Conv2D(x4_semantic_filters,(1,1),name='resize_filter_conv3_s',activation='relu')(bilinear_upsampling_3)
# bilinear_upsampling_3 = tf.keras.layers.experimental.preprocessing.Resizing(f4.shape[1], f4.shape[2], interpolation='bilinear')(f3)
# f_out3 = tf.keras.layers.Conv2D(f4_filters,(1,1),name='resize_filter_conv3')(bilinear_upsampling_3)

#F_out LAYER UPSAMPLING
bilinear_upsampling_4 = tf.keras.layers.experimental.preprocessing.Resizing(x4_semantic_shape_x_y[0], x4_semantic_shape_x_y[1], interpolation='bilinear')(x4_semantic)
x_semantic_out4 = tf.keras.layers.Conv2D(x4_semantic_filters,(1,1),name='resize_filter_conv4_s', activation='relu')(bilinear_upsampling_4)


''' Applying formulae for semi dense up skip connection '''
x_semantic_out_final1=tf.keras.layers.Add(name = 'final_semantic_layer_1')([x_semantic_out_in,x_semantic_out1,x_semantic_out2,x_semantic_out3,x_semantic_out4])
x_semantic_out_final = tf.keras.layers.Activation('sigmoid', name = 'semantic_final_layer')(x_semantic_out_final1)


#####################################################################################################################################
#####################################################################################################################################
#add_shape  of f4 output
x4_depth_shape_x_y = (240,320)
x4_depth_filters=x4_depth.shape[-1]

# BASE INPUT LAYER UPSAMPLING
bilinear_upsampling_in_d = tf.keras.layers.experimental.preprocessing.Resizing(x4_depth_shape_x_y[0], x4_depth_shape_x_y[1], interpolation='bilinear')(model.output)
x_depth_out_in=tf.keras.layers.Conv2D(x4_depth_filters,(1,1),name='resize_filter_conv_in_d',activation='relu')(bilinear_upsampling_in_d)

# FIRST LAYER BILINEAR UPsAMPLING
bilinear_upsampling_1_d = tf.keras.layers.experimental.preprocessing.Resizing(x4_depth_shape_x_y[0], x4_depth_shape_x_y[1], interpolation='bilinear')(x1_depth)
x_depth_out1 = tf.keras.layers.Conv2D(x4_depth_filters,(1,1),name='resize_filter_conv1_d',activation='relu')(bilinear_upsampling_1_d)

#SECOND LAYER BILINEAR UPSAMPLING
bilinear_upsampling_2_d = tf.keras.layers.experimental.preprocessing.Resizing(x4_depth_shape_x_y[0], x4_depth_shape_x_y[1], interpolation='bilinear')(x2_depth)
x_depth_out2 = tf.keras.layers.Conv2D(x4_depth_filters,(1,1),name='resize_filter_conv2_d',activation='relu')(bilinear_upsampling_2_d)

# 3rd LAYER UPSAMPLING
bilinear_upsampling_3_d = tf.keras.layers.experimental.preprocessing.Resizing(x4_depth_shape_x_y[0], x4_depth_shape_x_y[1], interpolation='bilinear')(x3_depth)
x_depth_out3 = tf.keras.layers.Conv2D(x4_depth_filters,(1,1),name='resize_filter_conv3_d',activation='relu')(bilinear_upsampling_3_d)
# bilinear_upsampling_3 = tf.keras.layers.experimental.preprocessing.Resizing(f4.shape[1], f4.shape[2], interpolation='bilinear')(f3)
# f_out3 = tf.keras.layers.Conv2D(f4_filters,(1,1),name='resize_filter_conv3')(bilinear_upsampling_3)

#F_out LAYER UPSAMPLING
bilinear_upsampling_4_d = tf.keras.layers.experimental.preprocessing.Resizing(x4_depth_shape_x_y[0], x4_depth_shape_x_y[1], interpolation='bilinear')(x4_depth)
x_depth_out4 = tf.keras.layers.Conv2D(x4_depth_filters,(1,1),name='resize_filter_conv4_d', activation='relu')(bilinear_upsampling_4_d)


''' Applying formulae for semi dense up skip connection '''
x_depth_out_final=tf.keras.layers.Add(name = 'depth_final_layer')([x_depth_out_in,x_depth_out1,x_depth_out2,x_depth_out3,x_depth_out4])

output = [x_semantic_out_final, x_depth_out_final]
model_final = tf.keras.models.Model(inputs = model.input, outputs = output)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# model_final.load_weights("/content/drive/MyDrive/Combined_42epoch/train_2.tf")
model_final.load_weights("/content/drive/MyDrive/depth_weights/train_11.tf")


Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2DTranspose object at 0x7fbee4079250> and <keras.layers.convolutional.Conv2DTranspose object at 0x7fbee3ec7bd0>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2DTranspose object at 0x7fbee4b52c90> and <keras.layers.merge.Add object at 0x7fbee3f3ffd0>).

Two checkpoint references resolved to different objects (<__main__.LSU object at 0x7fbee3f5acd0> and <keras.layers.merge.Add object at 0x7fbee3f5ad10>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2DTranspose object at 0x7fbee3eed090> and <keras.layers.convolutional.Conv2DTranspose object at 0x7fbee4248150>).

Two checkpoint references resolved to different objects (<keras.layers.convolutional.Conv2DTranspose object at 0x7fbee3ec7bd0> and <keras.layers.convolutional.Conv2DTranspose object at 0x7fbee3ec7a10>).


ValueError: ignored

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/nyu2_train.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/output.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
def ALE(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype = tf.float32)
    y_true = tf.clip_by_value(y_true, 0.003921, 1.0)
    y_pred = tf.clip_by_value(y_pred, 0.003921, 1.0)
    y_true = tf.math.log(y_true) / tf.math.log(10.0)
    y_pred = tf.math.log(y_pred) / tf.math.log(10.0)
    ale = tf.math.abs(y_true - y_pred)
    ale = tf.reduce_mean(ale)
    return ale

In [ ]:
def RMS(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype = tf.float32)
    rms = tf.math.pow((y_true - y_pred), 2)
    rms = tf.reduce_mean(rms)
    rms = tf.math.sqrt(rms)
    return rms

In [ ]:
def relative_error(y_true, y_pred, img):
    y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype = tf.float32)
    img = tf.convert_to_tensor(img, dtype = tf.float32)
    img = tf.clip_by_value(img, 1.0, 255.0)
    relative = (y_true - y_pred)
    relative = tf.math.abs(relative)
    relative = relative / img
    relative = tf.reduce_mean(relative)
    return relative

In [ ]:
def are(y_true, y_pred):
  y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
  y_pred = tf.convert_to_tensor(y_pred, dtype = tf.float32)
  y_true = tf.clip_by_value(y_true, 0.003921, 1.0)
  y_pred = tf.clip_by_value(y_pred, 0.003921, 1.0)
  error = (y_pred - y_true) / y_true
  error = tf.math.abs(error)
  error = error * 100
  error = tf.reduce_mean(error)
  return error

In [ ]:
ale = []
rms = []
rel = []
for i in range(len(img_paths)):
  img = cv2.imread(img_paths[i])
  img = cv2.resize(img,(320,240))
  img = img / 255.0
  img = np.expand_dims(img,axis=0)
  y_true = cv2.imread(depth_paths[i])
  y_true = cv2.resize(y_true,(320,240))
  # y_true = cv2.cvtColor(y_true, cv2.COLOR_BGR2GRAY)
  y_true = y_true / 255.0
  output = model_final.predict(img)
  y_pred = output[1][0][...,0]
  # print(y_pred)
  y_pred = cv2.cvtColor(y_pred, cv2.COLOR_GRAY2BGR)
  y_pred = y_pred / 255.0
  img = np.resize(img, (240, 320, 3))
  # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # img = img * 255.0
  relative_value = relative_error(y_true, y_pred, img)
  rms_value = RMS(y_true, y_pred)
  ale_value = ALE(y_true, y_pred)
  print('rel', relative_value)
  print('rms', rms_value)
  print('ale', ale_value)
  ale.append(ale_value)
  rms.append(rms_value)
  rel.append(relative_value)
  print('done', i)

Streaming output truncated to the last 5000 lines.
rel tf.Tensor(0.10997048, shape=(), dtype=float32)
rms tf.Tensor(0.13044068, shape=(), dtype=float32)
ale tf.Tensor(0.29497924, shape=(), dtype=float32)
done 6082
rel tf.Tensor(0.104424454, shape=(), dtype=float32)
rms tf.Tensor(0.11145804, shape=(), dtype=float32)
ale tf.Tensor(0.29713637, shape=(), dtype=float32)
done 6083
rel tf.Tensor(0.112656265, shape=(), dtype=float32)
rms tf.Tensor(0.11772235, shape=(), dtype=float32)
ale tf.Tensor(0.31628558, shape=(), dtype=float32)
done 6084
rel tf.Tensor(0.16579938, shape=(), dtype=float32)
rms tf.Tensor(0.18445477, shape=(), dtype=float32)
ale tf.Tensor(0.40060428, shape=(), dtype=float32)
done 6085
rel tf.Tensor(0.16512594, shape=(), dtype=float32)
rms tf.Tensor(0.1759434, shape=(), dtype=float32)
ale tf.Tensor(0.40436214, shape=(), dtype=float32)
done 6086
rel tf.Tensor(0.09918384, shape=(), dtype=float32)
rms tf.Tensor(0.1321058, shape=(), dtype=float32)
ale tf.Tensor(0.25657517, shape=

In [ ]:
ale = np.array(ale)
rel = np.array(rel)
rms = np.array(rms)

NameError: ignored

In [ ]:
!wget http://faust.is.tue.mpg.de/main/download

--2021-04-02 06:15:44--  http://faust.is.tue.mpg.de/main/download
Resolving faust.is.tue.mpg.de (faust.is.tue.mpg.de)... 192.124.27.130
Connecting to faust.is.tue.mpg.de (faust.is.tue.mpg.de)|192.124.27.130|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://faust.is.tue.mpg.de/login [following]
--2021-04-02 06:15:45--  http://faust.is.tue.mpg.de/login
Reusing existing connection to faust.is.tue.mpg.de:80.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download’

download                [ <=>                ]   5.63K  --.-KB/s    in 0s      

2021-04-02 06:15:45 (551 MB/s) - ‘download’ saved [5765]

